In [ ]:
import sys
import localSettings as ls

In [ ]:
main_path = ls.main_path
sys.path.append(main_path)

In [ ]:
import plotter
import importlib
importlib.reload(plotter)
import uproot
import matplotlib.pylab as pylab
import numpy as np
import math
from sklearn.model_selection import train_test_split
import pickle
import xgboost as xgb
import nue_booster 
importlib.reload(nue_booster)
import awkward

params = {
    'axes.labelsize': 'x-large',
    'axes.titlesize': 'x-large',
    'xtick.labelsize': 'x-large',
    'ytick.labelsize': 'x-large'
}
pylab.rcParams.update(params)

In [ ]:
tree = "NeutrinoSelectionFilter"

OLD = 'data_bnb_mcc9.1_v08_00_00_25_reco2_C1_beam_good_reco2_5e19'+ls.APPEND
NEW = 'data_bnb_peleeTuple_unblinded_uboone_v08_00_00_42_run1_C1'+ls.APPEND

uold = uproot.open('root_files/0304/nuepresel/'+OLD+".root")[ls.fold][tree]
unew = uproot.open('root_files/0414/'+NEW+".root")[ls.fold][tree]

uproot_v = [uold, unew]

variables = [
    "shr_dedx_Y", "shr_bkt_pdg", "p", "pt", "selected", "nu_pdg", "shr_theta",
    "slpdg", "trk_score_v", "backtracked_pdg", # modified from shr_score_v
    "shr_pfp_id_v", "category",
    "shr_tkfit_dedx_U","shr_tkfit_dedx_V","shr_tkfit_dedx_Y",
    "shr_tkfit_gap10_dedx_U","shr_tkfit_gap10_dedx_V","shr_tkfit_gap10_dedx_Y",
    "shr_tkfit_2cm_dedx_U","shr_tkfit_2cm_dedx_V","shr_tkfit_2cm_dedx_Y",
    "topological_score",
    #"shr_energy_tot", 
    "trk_energy_tot", "shr_hits_tot", "ccnc", "trk_chipr",
    "trk_bkt_pdg", "hits_ratio", "n_tracks_contained", 
    "crtveto","crthitpe","_closestNuCosmicDist",
    "NeutrinoEnergy2",
    #"run","sub","evt",
    "CosmicIP","CosmicDirAll3D","CosmicIPAll3D",
    "nu_flashmatch_score","best_cosmic_flashmatch_score","best_obviouscosmic_flashmatch_score",
    #"trk_pfp_id",
    "shrmoliereavg","shrmoliererms",
    "shr_tkfit_npointsvalid","shr_tkfit_npoints", # fitted vs. all hits for shower
    "shrclusfrac0","shrclusfrac1","shrclusfrac2", # track-fitted hits / all hits
    "trkshrhitdist2", # "trkshrhitdist0","trkshrhitdist1", distance between track and shower in 2D
    "shrsubclusters0","shrsubclusters1","shrsubclusters2", # number of sub-clusters in shower
    "trk_llr_pid_score_v", # trk-PID score
    "trk_energy_proton_v", # track energy under proton hyp
    "trk_calo_energy_y_v", # track calo energy
    #"pi0_energy2_Y", # pi0 tagger variables
    "_opfilter_pe_beam", "_opfilter_pe_veto", # did the event pass the common optical filter (for MC only)
    "reco_nu_vtx_sce_x","reco_nu_vtx_sce_y","reco_nu_vtx_sce_z",
    "nproton", "nu_e", "n_showers_contained", "shr_distance", "trk_distance",
    "hits_y", "shr_pz", "shr_energy", "shr_dedx_U", "shr_dedx_V", "shr_phi", "trk_phi", "trk_theta",
    "shr_tkfit_dedx_U", "shr_tkfit_dedx_V", "run", "sub", "evt", "nproton", "trk_pid_chipr_v",
    "trk_len", "mc_pdg", "slnunhits", "slnhits", "shr_score", "trk_score", "trk_hits_tot",
    "true_e_visible", "matched_E", "shr_bkt_E", "trk_bkt_E", "trk_energy", "tksh_distance", "tksh_angle",
    "npi0","npion","pion_e","muon_e","pi0truth_elec_etot",
    "pi0_e", "shr_energy_tot_cali", "shr_dedx_Y_cali", "evnunhits", "nslice", "interaction",
    "slclustfrac", "reco_nu_vtx_x", "reco_nu_vtx_y", "reco_nu_vtx_z","contained_fraction",
    "secondshower_Y_nhit","secondshower_Y_vtxdist","secondshower_Y_dot","secondshower_Y_dir","shrclusdir2",
    "shr_tkfit_nhits_Y","shr_tkfit_nhits_U","shr_tkfit_nhits_V",
    "shr_tkfit_2cm_nhits_Y","shr_tkfit_2cm_nhits_U","shr_tkfit_2cm_nhits_V",
    "shr_tkfit_gap10_nhits_Y","shr_tkfit_gap10_nhits_U","shr_tkfit_gap10_nhits_V",
    "pfnhits","pfnunhits"
]
#make the list unique
variables = list(set(variables))
print(variables)

#if not ISRUN3:
variables.remove("_closestNuCosmicDist")
variables.remove("crtveto")
variables.remove("crthitpe")
    
NEWVARS = ['run', 'sub', 'evt', 'reco_e', 'bdt_pi0_np', 'bdt_nonpi0_np', 'bdt_bkg_0p',
           'shr_score','tksh_distance','tksh_angle','shr_tkfit_dedx_max','trkfit','trkpid',
           'subcluster','shrmoliereavg','trkshrhitdist2','hits_ratio',
           'secondshower_Y_nhit','secondshower_Y_vtxdist','secondshower_Y_dot','anglediff_Y',
           'CosmicIPAll3D','CosmicDirAll3D']

dold = uold.pandas.df(variables, flatten=False)
dnew = unew.pandas.df(NEWVARS, flatten=False)

In [ ]:
df_v = [dold]
#df_v = [lee,mc,nue,ext,data,dirt]

for i,df in enumerate(df_v):
    up = uproot_v[i]
    trk_llr_pid_v = up.array('trk_llr_pid_score_v')
    trk_energy_proton_v = up.array('trk_energy_proton_v')
    trk_calo_energy_y_v = up.array('trk_calo_energy_y_v')
    trk_id = up.array('trk_id')-1 # I think we need this -1 to get the right result
    trk_llr_pid_v_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_llr_pid_v,trk_id)])
    trk_energy_proton_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_energy_proton_v,trk_id)])
    trk_calo_energy_y_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_calo_energy_y_v,trk_id)])
    df['trkpid'] = trk_llr_pid_v_sel
    df['protonenergy'] = trk_energy_proton_sel
    df['trackcaloenergy'] = trk_calo_energy_y_sel
    df['subcluster'] = df['shrsubclusters0'] + df['shrsubclusters1'] + df['shrsubclusters2']
    df['trkfit'] = df['shr_tkfit_npointsvalid'] / df['shr_tkfit_npoints']
    df['anglediff_Y'] = np.abs(df['secondshower_Y_dir']-df['shrclusdir2'])
    df['trkthetacos'] = np.cos(df['trk_theta'])


In [ ]:
for i,df in enumerate(df_v):
    df['shr_tkfit_nhits_tot'] = (df['shr_tkfit_nhits_Y']+df['shr_tkfit_nhits_U']+df['shr_tkfit_nhits_V'])
    df['shr_tkfit_dedx_avg'] = (df['shr_tkfit_nhits_Y']*df['shr_tkfit_dedx_Y'] + df['shr_tkfit_nhits_U']*df['shr_tkfit_dedx_U'] + df['shr_tkfit_nhits_V']*df['shr_tkfit_dedx_V'])/df['shr_tkfit_nhits_tot']
    df['shr_tkfit_2cm_nhits_tot'] = (df['shr_tkfit_2cm_nhits_Y']+df['shr_tkfit_2cm_nhits_U']+df['shr_tkfit_2cm_nhits_V'])
    df['shr_tkfit_2cm_dedx_avg'] = (df['shr_tkfit_2cm_nhits_Y']*df['shr_tkfit_2cm_dedx_Y'] + df['shr_tkfit_2cm_nhits_U']*df['shr_tkfit_2cm_dedx_U'] + df['shr_tkfit_2cm_nhits_V']*df['shr_tkfit_2cm_dedx_V'])/df['shr_tkfit_2cm_nhits_tot']
    df['shr_tkfit_gap10_nhits_tot'] = (df['shr_tkfit_gap10_nhits_Y']+df['shr_tkfit_gap10_nhits_U']+df['shr_tkfit_gap10_nhits_V'])
    df['shr_tkfit_gap10_dedx_avg'] = (df['shr_tkfit_gap10_nhits_Y']*df['shr_tkfit_gap10_dedx_Y'] + df['shr_tkfit_gap10_nhits_U']*df['shr_tkfit_gap10_dedx_U'] + df['shr_tkfit_gap10_nhits_V']*df['shr_tkfit_gap10_dedx_V'])/df['shr_tkfit_gap10_nhits_tot']
    df.loc[:,'shr_tkfit_dedx_max'] = df['shr_tkfit_dedx_Y']
    df.loc[(df['shr_tkfit_nhits_U']>df['shr_tkfit_nhits_Y']),'shr_tkfit_dedx_max'] = df['shr_tkfit_dedx_U']
    df.loc[(df['shr_tkfit_nhits_V']>df['shr_tkfit_nhits_Y']) & (df['shr_tkfit_nhits_V']>df['shr_tkfit_nhits_U']),'shr_tkfit_dedx_max'] = df['shr_tkfit_dedx_V']

In [ ]:
INTERCEPT = 0.0
SLOPE = 0.83

# define some energy-related variables
for i,df in enumerate(df_v):
    df["reco_e"] = (df["shr_energy_tot_cali"] + INTERCEPT) / SLOPE + df["trk_energy_tot"]
    df["reco_e_qe"] = 0.938*((df["shr_energy"]+INTERCEPT)/SLOPE)/(0.938 - ((df["shr_energy"]+INTERCEPT)/SLOPE)*(1-np.cos(df["shr_theta"])))
    df["reco_e_rqe"] = df["reco_e_qe"]/df["reco_e"]

In [ ]:
# variables to be trained on
TRAINVAR = ["shr_score","tksh_distance","tksh_angle",
            "shr_tkfit_dedx_max","trkfit","trkpid",
            "subcluster","shrmoliereavg",
            "trkshrhitdist2","hits_ratio",
            "secondshower_Y_nhit","secondshower_Y_vtxdist","secondshower_Y_dot","anglediff_Y",
            "CosmicIPAll3D","CosmicDirAll3D"]

In [ ]:
LABELS = ['pi0','nonpi0']

for label, bkg_query in zip(LABELS, nue_booster.bkg_queries):
    with open(ls.pickle_path+'booster_%s_0304_extnumi.pickle' % label, 'rb') as booster_file:
        booster = pickle.load(booster_file)
        print((ls.pickle_path+'booster_%s_0304_extnumi.model' % label), booster.best_iteration)
        booster.save_model( (ls.pickle_path+'booster_%s_0304_extnumi.model' % label) )
        dold[label+"_score"] = booster.predict(
            xgb.DMatrix(dold[TRAINVAR]),
            ntree_limit=booster.best_iteration)
        dnew[label+"_score"] = booster.predict(
            xgb.DMatrix(dnew[TRAINVAR]),
            ntree_limit=booster.best_iteration)

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
tablevars = ["run","sub","evt","reco_e","pi0_score","nonpi0_score"]+TRAINVAR
dold.query("run==5215 & evt==2625")[tablevars]

In [ ]:
tablevarsnew = ["pi0_score","nonpi0_score"]+NEWVARS
dnew.query("run==5215 & evt==2625")[tablevarsnew]

In [ ]:
dmrg = pd.merge(dold, dnew, on=['run','sub','evt'], how='left', validate="1:1")
#print(dmrg.columns.tolist())
dmrg.query("abs(pi0_score_x-bdt_pi0_np)>0.000001")[["run","sub","evt","reco_e_x","reco_e_y","pi0_score_x","pi0_score_y","bdt_pi0_np","nonpi0_score_x","nonpi0_score_y","bdt_nonpi0_np"]]

#dmrg.query("abs(pi0_score-bdt_pi0_np)>0.000001")[["shr_score_x","tksh_distance_x","shr_score_y","tksh_distance_y"]]
#dmrg.query("abs(pi0_score-bdt_pi0_np)>0.000001")[["tksh_angle_x","shr_tkfit_dedx_max_x","tksh_angle_y","shr_tkfit_dedx_max_y"]]
#dmrg.query("abs(pi0_score-bdt_pi0_np)>0.000001")[["trkfit_x","trkpid_x","trkfit_y","trkpid_y"]]
#dmrg.query("abs(pi0_score_x-bdt_pi0_np)>0.000001")[["subcluster_x","shrmoliereavg_x","subcluster_y","shrmoliereavg_y"]]
#dmrg.query("abs(pi0_score-bdt_pi0_np)>0.000001")[["trkshrhitdist2_x","hits_ratio_x","trkshrhitdist2_y","hits_ratio_y"]]
#dmrg.query("abs(pi0_score-bdt_pi0_np)>0.000001")[["secondshower_Y_nhit_x","secondshower_Y_vtxdist_x","secondshower_Y_nhit_y","secondshower_Y_vtxdist_y"]]
#dmrg.query("abs(pi0_score-bdt_pi0_np)>0.000001")[["secondshower_Y_dot_x","anglediff_Y_x","secondshower_Y_dot_y","anglediff_Y_y"]]
#dmrg.query("abs(pi0_score-bdt_pi0_np)>0.000001")[["CosmicIPAll3D_x","CosmicDirAll3D_x","CosmicIPAll3D_y","CosmicDirAll3D_y"]]

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(5, 5))
var1 = dmrg['pi0_score_x']-dmrg['bdt_pi0_np']
varbls = [var1]
plt.hist(varbls, bins=1001, range=(-0.05,0.05), #weights=[normOne(v) for v in varbls],
         histtype='step',linestyle='solid', linewidth=1.2, #,color=colors[:-1]
         stacked=False, label=['all BNB','after loose selection'])
plt.title('Run1 BNB 4e19, after nue preselection')
plt.xlabel('pi0_score-bdt_pi0_np')
#plt.yscale('log')
#plt.grid(axis='y',linestyle=':')
#plt.legend(loc=1)


In [ ]:
for v in TRAINVAR:
    plt.figure(figsize=(5, 5))
    var1 = (dmrg[v+'_x']-dmrg[v+'_y'])/dmrg[v+'_x']
    varbls = [var1]
    plt.hist(varbls, bins=10001, range=(-0.5,0.5), #weights=[normOne(v) for v in varbls],
             histtype='step',linestyle='solid', linewidth=1.2, #,color=colors[:-1]
             stacked=False, label=['all BNB','after loose selection'])
    plt.title('Run1 BNB 4e19, after nue preselection')
    plt.xlabel(v+" diff")
    plt.yscale('log')
    plt.ylim(0.1,2000) 